# Genrating Images
In this notebook, we will use a trained GAN to generate an experimental image from simulated data.

In [ ]:
import tensorflow as tf
from models import unet_generator
from generate_image import generate_image, GAN_image_folder
import os, shutil

In [ ]:
parent_dir = "D:/stem-learning/"
sim_dir = parent_dir + "data/WSe/data_for_gan/simulation/sim_pristine_0.1_noised/"
identifier = "20220525_unet_dist_gen_fft_10_pristine_gaussian_0.1"

fine_size, stride = 256, 256
num_channels=1


model_type = "exp"
input_file = sim_dir + "sim_images_stack0000_000.tiff"
fine_size = 256
stride= 256
avg=False
plot=True
save_data=True
save_dir=parent_dir
fname='generated_input.tiff'

save_dir = parent_dir + "data/WSe/data_for_gan/generated/GANNED_SIM/sim_abberation_{}/".format(identifier)

input_dir = sim_dir

########################################################################################################################
data_folder_with_labels = parent_dir + "data/WSe/data_for_gan/simulation/data_folders_with_labels/"
input_folder = save_dir
target_folder = parent_dir + "data/WSe/data_for_gan/generated/GANNED_DATA_FOLDER/data_folder_{}".format(identifier)

In [ ]:
generator_exp = unet_generator(num_channels, 1, "instancenorm")
generator_sim = unet_generator(1, num_channels, "instancenorm")
model = generator_exp if model_type == "exp" else generator_sim

In [ ]:
checkpoint_path = parent_dir + "cycle_gan_results/checkpoints/checkpoint_{}".format(identifier)
log_data_fn = "{}/data_{}.p".format(checkpoint_path, identifier)

ckpt = tf.train.Checkpoint(generator_exp=generator_exp,
                           generator_sim=generator_sim)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=3)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    #cpath = ckpt_manager.checkpoints[-2]
    cpath = ckpt_manager.latest_checkpoint
    print("loading checkpoint ", cpath)
    ckpt.restore(cpath)
    print ('Latest checkpoint restored!')

In [ ]:
img = generate_image(model, input_file, fine_size, stride, avg, plot, save_data, save_dir,fname)

If we want to make batch images:

In [ ]:
GAN_image_folder(model, input_dir, save_dir, fine_size, stride, avg)

In [ ]:
def make_folder(data_folder_with_labels, input_folder, target_folder):
    shutil.copytree(data_folder_with_labels, target_folder, dirs_exist_ok=True)
    for img_file, save_dir in zip(os.listdir(input_folder), os.listdir(target_folder)):
        full_src = os.path.join(input_folder, img_file)
        full_dst = os.path.join(target_folder, save_dir, "input.tiff")
        shutil.copy(full_src, full_dst)

In [ ]:
make_folder(data_folder_with_labels, input_folder, target_folder)